## Module submission header
### Submission preparation instructions 
_Completion of this header is mandatory, subject to a 2-point deduction to the assignment._ Only add plain text in the designated areas, i.e., replacing the relevant 'NA's. You must fill out all group member Names and Drexel email addresses in the below markdown list, under header __Module submission group__. It is required to fill out descriptive notes pertaining to any tutoring support received in the completion of this submission under the __Additional submission comments__ section at the bottom of the header. If no tutoring support was received, leave NA in place. You may as well list other optional comments pertaining to the submission at bottom. _Any distruption of this header's formatting will make your group liable to the 2-point deduction._

### Module submission group
- Group member 1
    - Name: Johnny Myers
    - Email: jtm372@drexel.edu
- Group member 2
    - Name: Hyung Wook Choi
    - Email: hc685@drexel.edu
- Group member 3
    - Name: Naimish Bizzu
    - Email: nb924@drexel.edu
- Group member 4
    - Name: Sushanth Reddy Sangannagari
    - Email: ss5228@drexel.edu

### Additional submission comments
- Tutoring support received: NA
- Other (other): NA

# Assignment Group 3

## Module C _(59 points)_

In this exercise we'll build and expand a cheese and wine pairings database with the high-level goal of linking wine varieties and recipe ingredients.

A Google search on food and wine pairings turned up a literature Ph.D. candidate's blog post:
+ http://sedimentality.com/drinking-wine/list-of-wine-and-food-pairings/

This post is interesting, because instead of pairing dishes with wine vintages or brands, it pairs ingredients with wine varieties, tagging ingredients by several categories. Thus, we could pull the data from this website to join wine varieties from a large wine reviews database to ingredients in a large recipes database. Your goal in this section of the assignment is to collect the ingredient-variety-category data.

__C1.__ _(2 pts)_ Why are we allowed to scrape this data out of the website?

Explain what information is posted (or not posted) on this domain, allowing us to definitely move forward with this exercise. Use the markdown cell below.

<font color=red>The website is run as a Wordpress blog. Wordpress permits a copy for personal use from their websites. Additionally, on Wordpress, users have the option to copy all content from a website for personal use. Because the blog does not specifically say that we cannot use the data for personal purposes, we can default to Wordpress's rules and scrape data out of the website. No information counter to Wordpress's rules are posted on the domain.</font>

__C2.__ _(2 pts)_ Download the blog post using the `requests` module and store the html response.

In [20]:
import pandas as pd
import urllib
import csv
import re
import json
import requests
from bs4 import BeautifulSoup as bs
from pprint import pprint
import collections
from collections import OrderedDict

In [21]:
# Get the URL for the dataset
url = "http://sedimentality.com/drinking-wine/list-of-wine-and-food-pairings/"

#request the url and store as a requests.models.Response
html = requests.get(url)

#beautifulsoup the html.text  file
soup = bs(html.text, 'html.parser')

#prettyprint the soup
pprint(soup)


<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html lang="en-US">
<head profile="http://gmpg.org/xfn/11">
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<link href="http://sedimentality.com/xmlrpc.php" rel="pingback"/>
<title>List of wine and food pairings | Sedimentality</title>
<!-- All In One SEO Pack 3.4.2[399,452] -->
<meta content="&quot;What wine goes with Captain Crunch?&quot; -George Karlin Pairing wines and food is too often stressed about (and debated). True, in these days we no longer abide by rules like &quot;ONLY white with fish&quot; (ever had a coriander and coffee encrusted ahi with Syrah? Amazing!) but we are often confused about how to pair certain flavors and elements of a dish. The following are some guidelines for pairing a few common wines. Remember, pairing is a funny thing, because every dish

__C3.__ _(4 pts)_ Navigate to the same URL in your web browser, and identify where the target data are by inspecting the html (with your eyes) and matching up tags with the target content.

In the markdown cell below, describe the tags that you will need to extract the data from. Provide as much information as is necessary on the nesting of the document so as to be able to describe the location of the target data for parsing.

<font color=red>The information we are looking for is:     
    - Name of wine: <h2>
    - Cheese/nuts: <p>
    - Meat/poultry: <p>
    - Seafood: <p>
    - Fruits and Veggies: <p>
    - Herbs and Spices: <p>
    - Sauces: <p>
    - Desserts: <p>
    

The content can be found: "body class / div class="row" / div id = "content" / div class="top-content" / div class="post_info_wrap""

__C4.__ _(6 pts)_ Parse the html text with BeautifulSoup in preparation for scraping. Then, print out the target data. \[__Hint__: use `BeautifulSoup`'s `.find_all(tag_list)` method to iterate through the tags identified in __C3__. From there, check to see if a tag has the data of interest by using the `.find(tag)` method. \]

In [22]:
# use soup.find_all to find the <div> tags relevant.
# we get a lot of results, but the results we are interested in come from the 16th div tag.
# so soup.findall("div")[16]
winediv = soup.find_all("div")[16]

pprint(winediv)

<div class="post_info_wrap"><div class="shareaholic-canvas" data-app="share_buttons" data-app-id-name="page_above_content" data-link="http://sedimentality.com/list-of-wine-and-food-pairings/" data-title="List of wine and food pairings" style="display:none;"></div>
<figure class="wp-block-image"><img alt="" class="wp-image-8486" sizes="(max-width: 640px) 100vw, 640px" src="http://sedimentality.com/wp-content/uploads/2019/10/wine-791133_640.jpg" srcset="http://sedimentality.com/wp-content/uploads/2019/10/wine-791133_640.jpg 640w, http://sedimentality.com/wp-content/uploads/2019/10/wine-791133_640-300x200.jpg 300w"/></figure>
<p style="text-align: center;"><em>“What wine goes with Captain Crunch?” -George Karlin</em></p>
<p style="text-align: justify;">Pairing wines and food is too often stressed about (and debated). True, in these days we no longer abide by rules like “ONLY white with fish” (ever had a coriander and coffee encrusted ahi with Syrah? Amazing!) but we are often confused abo

__C5.__ _(3 pts)_ Interpret the output from the previous part to make a plan for how to split the target data up with regular expressions to create a structured data object. Describe any structure that you'll use in the markdown cell, below.

<font color=red>The wine names and related ingredients are in a massive list as strings. The best way to split up the target data is to use regex to create a structured object. I think building a dictionary using regex is the best way to go about organizing the data. Additionally, putting the data in a JSON object would make it cleaner and organize the information. We can use the "/strong" tag to split the categories (fruits, nuts, etc.) from their values, and we can also use the ":" sign to extract the categories with the values.</font>

__C6.__ _(2 pts)_ Create an associative array (dictionary) data structure that can hold the ingredients in their categories, by wine variety. Use a default dictionary with its default value as a dictionary.

In [23]:
#Goal: create dictionary that holds ingredients in category by wine. Use default dict with default value as dictionary.

#1. We need to find all the <p> tags in the html 
ptag_wine = winediv.find_all('p')
list_ingr = []

#function: for each value, find the strong tag, and append the values after the strong tag.
for tag in ptag_wine:
    
    #The categories are signalled with a </strong> tag. Other values with a <p> don't have the categories in it.
    if tag.find('strong'):  
    
        #add the value to the list
        list_ingr.append(tag.text) 
        
print(list_ingr) #prints out all the categories of ingredients and their values.
        
# 2. Same process as above
# Find all the h2 values (the types of wine), also a blank list "Variety" representing the varities of wines.
variety_wines = winediv.find_all('h2')
Variety = []

#append all the wine names to the blank list
for wine in variety_wines:
     Variety.append(wine.text)
        
print(Variety) #prints out all the varieties of wines.

#Now, we have a list of ingredients, and a list of wines. 

['Cheese/nuts: feta, goat cheese, pine nuts', 'Meat/poultry: chicken, turkey, pork', 'Seafood: fatty white fish, oysters, scallops, lobster, shrimp, sushi', 'Fruits and Veggies: citrus, green apple, asparagus', 'Herbs and Spices: chives, tarragon, cilantro', 'Sauces: citrus and light cream sauces', 'Desserts: sorbet, key lime pie, meringue, mango', 'Cheese/nuts:\xa0mild, semi-soft cheeses with unoaked Chardonnay; asiago, havarti, Stilton or other blue-veined cheeses with oaky Chardonnay; almonds and nearly any toasted nut', 'Meat/poultry: veal, chicken, pork', 'Seafood: halibut, shrimp, crab, lobster', 'Fruits and Veggies: potato, apple, squash, mango', 'Herbs and Spices: tarragon, sesame, basil', 'Sauces: cream sauces, pesto', 'Desserts: banana bread, vanilla pudding', 'Cheese/nust: Havarti, gouda, candied walnuts or pecans', 'Meat/poultry: smoked sausage, duck, foie gras', 'Seafood: sea bass, trout', 'Fruits and Veggies: apricots, chili peppers, pears', 'Herbs and Spices: rosemary, g

__C7.__ _(7 pts)_ Complete the web-scraping script's loop in __C4__ and store the extracted data in the data structure you initialized in __C6__. Print the stored data to confirm your output. Note you'll need to use the regular expresions module to complete this section.

In [30]:
WineDictionary = []

#there are 7 total types of wine, so we'll do this function 7 times.
numberWines = list(range(1,8))

#for each wine, build a subdictionary and 
for n in numberWines:
    
    #let's start a new dictionary for each subwine. At the end we will append all the winesubdictionaries to the WineDictionary
    WineSubDictionary = {}
    
    #to WineSubDictionary's Name key, add each name of the wine. 
    WineSubDictionary["Name"] = [name for name in Variety][n-1]
    
    #to WineSubDictionary's CheeseAndNuts key, add each value of cheese and nuts. 
    WineSubDictionary["CheeseAndNuts"] = list_ingr[len(numberWines)*(n-1)].split(":")[1]
    
    #to WineSubDictionary's MeatAndPoultry key, add each value of meat and poultry. 
    WineSubDictionary["MeatAndPoultry"] = list_ingr[len(numberWines)*(n-1)+1].split(":")[1]
    
    #to WineSubDictionary's Seafood key, add each value of seafood.
    WineSubDictionary["Seafood"] = list_ingr[len(numberWines)*(n-1)+2].split(":")[1]
    
    #to WineSubDictionary's FruitsAndVeggies key, add each value of fruit and veggies.
    WineSubDictionary["FruitsAndVeggies"] = list_ingr[len(numberWines)*(n-1)+3].split(":")[1]
    
    #to WineSubDictionary's HerbsAndSpcies key, add each value of herbs and spices.
    WineSubDictionary["HerbsAndSpices"] = list_ingr[len(numberWines)*(n-1)+4].split(":")[1]
    
    #to WineSubDictionary's Sauces key, add each value of Sauces.
    WineSubDictionary["Sauces"] = list_ingr[len(numberWines)*(n-1)+5].split(":")[1]
    
    #to WineSubDictionary's Desserts key, add each value of desserts.
    WineSubDictionary["Desserts"] = list_ingr[len(numberWines)*(n-1)+6].split(":")[1]
       
        
    #note that one problem is that the dictionary keys are sorted alphabetically, which is not super helpful.
    #We should put name first.

    order = ['Name', 'CheeseAndNuts', 'Desserts', "FruitsAndVeggies", "HerbsAndSpices", 'MeatAndPoultry', 'Sauces', 'Seafood']

    WineSubDictionary = OrderedDict((k, WineSubDictionary[k]) for k in order)
    
    #append all subdictionaries
    WineDictionary.append(WineSubDictionary)
    
#prettyprint that bad boy
pprint(WineDictionary)

[OrderedDict([('Name', 'Sauvignon Blanc'),
              ('CheeseAndNuts', ' feta, goat cheese, pine nuts'),
              ('Desserts', ' sorbet, key lime pie, meringue, mango'),
              ('FruitsAndVeggies', ' citrus, green apple, asparagus'),
              ('HerbsAndSpices', ' chives, tarragon, cilantro'),
              ('MeatAndPoultry', ' chicken, turkey, pork'),
              ('Sauces', ' citrus and light cream sauces'),
              ('Seafood',
               ' fatty white fish, oysters, scallops, lobster, shrimp, '
               'sushi')]),
 OrderedDict([('Name', 'Chardonnay'),
              ('CheeseAndNuts',
               '\xa0mild, semi-soft cheeses with unoaked Chardonnay; asiago, '
               'havarti, Stilton or other blue-veined cheeses with oaky '
               'Chardonnay; almonds and nearly any toasted nut'),
              ('Desserts', ' banana bread, vanilla pudding'),
              ('FruitsAndVeggies', ' potato, apple, squash, mango'),
              ('Her

__C8.__ _(2 pts)_ Write your data out to a json file. Name this file: `./data/wine_ingredient-pairings.json`.

Note: since `defaultdicts` are not json serialiable (you can't make a file out of them), you will have to coerce (type cast) your final data to a standard Python dictionary to finish the job.

In [31]:
#json.dump it
json.dump(WineDictionary, open("./data/wine_ingredient-pairings.json", "w"))

__C9.__ _(3 pts)_ What limitations did you observe in this data, and in your extraction of this data?

Were there any issues either with this data's content, volume, its veracity or accuracy? Were you able to get all of the data present? Did you wind up getting some data that you didn't want or need, or maybe not in the right form? Take a look at the wine and categories keys—are they consistent? Be as specific as possible!

<font color=red>The biggest challenge was extracting data from the HTML file. Specifically, the way the HTML file arranged data - the ingredients and the wine types - was difficult to pull. If there were no "/strong" tags or no colons, this would be a much more difficult task. Additionally, figuring out that the value was the 16th item in the list took a long time. We got some extraneous data, which we needed to filter out to get the important values. The wine and category keys are not the most consistent. We have some weird values ("\xa0mild" in the second semidict), and pulling the data perfectly will take a bit of engineering or some manual efforts. The final thing is that the semidicts are out of order. The order is Cheese and Nuts, deserts, herbs and spices, meat and poultry, name, sauces, seafood... when name should come first. We will need to manually fix that.</font>

__C10.__ _(2 pts)_ Supposing we were going to rebuild the scraped wine-food pairings data as a gold standard through a survey. Are there any changes to allowable food types or to the manner in which the data are organized that might be beneficial to the wine recommendation task?

<font color=red>There are a ton of ways in which we can split up this data. We can subset our categories even further - meat versus poultry, fruits and veggies separated, and cheese and nuts separated. We can also split up the types of wine by red versus white.</font>

__C11.__ _(3 pts)_ Download and familiarize yourself with the cuisine prediction dataset: https://www.kaggle.com/kaggle/recipe-ingredients-dataset (it's only necessary to download the `train.json` file and work with that).
We're going to enrich our current data by adding results from this dataset.

In [33]:
#get the recipes from the json object
rec = json.load(open("./data/train.json/train.json", "r"))

#prettyprint the first five values to see what we're working with.
pprint(rec[0:4])

#how long is the full thing? This might get messy...
print("Length: " + str(len(rec)))

#turn this dataset into a pandas dataframe.
dfrec = pd.DataFrame(rec)
dfrec.head() #head of dataframe -- looks good. We have number, ID, cuisine, and ingredients stored as a list.

sortedrec = dfrec.sort_values(by = ["id"]) #let's sort by ID, just for fun.
sortedrec.head()

[{'cuisine': 'greek',
  'id': 10259,
  'ingredients': ['romaine lettuce',
                  'black olives',
                  'grape tomatoes',
                  'garlic',
                  'pepper',
                  'purple onion',
                  'seasoning',
                  'garbanzo beans',
                  'feta cheese crumbles']},
 {'cuisine': 'southern_us',
  'id': 25693,
  'ingredients': ['plain flour',
                  'ground pepper',
                  'salt',
                  'tomatoes',
                  'ground black pepper',
                  'thyme',
                  'eggs',
                  'green tomatoes',
                  'yellow corn meal',
                  'milk',
                  'vegetable oil']},
 {'cuisine': 'filipino',
  'id': 20130,
  'ingredients': ['eggs',
                  'pepper',
                  'salt',
                  'mayonaise',
                  'cooking oil',
                  'green chilies',
                  'grilled chicken bre

,id,cuisine,ingredients
31571,0,spanish,"[mussels, ground black pepper, garlic cloves, ..."
25290,1,mexican,"[tomatoes, diced red onions, paprika, salt, co..."
23447,2,french,"[chicken broth, truffles, pimentos, green pepp..."
19464,3,chinese,"[fresh ginger, sesame oil, frozen peas, cooked..."
19690,4,italian,"[orange peel, cookies, vanilla ice cream, gran..."


__C12.__ _(8 pts)_ Expand each wine variety's list of foods to include any co-appearing in the recipes found in the above Kaggle dataset. Make sure to record how often ingredients in the Kaggle recipes occured alongside foods in the wine variety list. Store this information in another nested `defaultdict`, with the following structure:

```
wines = {
    wine: {
        food_category: {
            pairing: Counter(),
            ...
        },
        ...
    },
    ...
}
```

In [34]:
#WineDictionary is a list
Wines = {}

#nested for loops: 
for wine_catg in WineDictionary:
    
    #for each wine, 
    for i in range(len(WineDictionary)):
        
        #create a new dictionary
        Food = {}
        
        #First, for each key in WineDictionary
        for akey in WineDictionary[i].keys():
            
            # Strip based on a space and a comma. First, strip the space. Then, split the comma and space (", ")
            # Add a new list called Food Counter that counts the number of times a food
            # is included in the data. 
            
            foodlist = wine_catg[akey].strip().split(", ")
            foodcount = []
            
            #for each food in the list, add ingredients
            for food in foodlist:
                n = []
                
                #
                for i in rec:
                    if food in i['ingredients']:
                        n.append(True)
                    else:
                        n.append(False)
                        
                # sum up count of ingredients for each wine so we can get a good sense of how many times
                # an ingredient is used. 
                foodcount.append((food, sum(n)))
                
                   
            Food[akey] = foodcount
            
            #remove the value "Name" from the dictionary
            Food.pop("Name", None)
            
            #food 
            Wines[wine_catg['Name']] = Food
            
#prettyprint Cabernet Sauvignon
pprint(Wines['Cabernet Sauvignon '])

{'CheeseAndNuts': [('cheddar', 0), ('gorgonzola', 31), ('walnuts', 114)],
 'Desserts': [('bittersweet chocolate', 140)],
 'FruitsAndVeggies': [('black cherries', 1),
                      ('tomatoes', 3058),
                      ('broccoli', 220)],
 'HerbsAndSpices': [('rosemary', 107), ('juniper', 0), ('lavender', 4)],
 'MeatAndPoultry': [('venison', 6), ('rib eye', 0), ('beef stew', 1)],
 'Sauces': [('brown sauce', 1), ('tomato sauce', 865)],
 'Seafood': [('grilled ahi tuna', 0)]}


__C13.__ _(2 pts)_ How would you create the above structure for `wines` using nested a nested `defaultdict`?

<font color=red>Instead of searching the columns in the dataframe, we can use a nested defaultdict to search for wines. Defaultdicts would be faster to use.</font>

__C14.__ _(6 pts)_ We'd also like to incorporate a wine review dataset. Download and familiarize yourself with the dataset, found at: https://www.kaggle.com/zynicide/wine-reviews. Specifically, utilize the file: `winemag-data-130k-v2.csv` to link each of the scraped wines found in `wines` to the specific names of the wines which are of that variety (i.e., for the Sauvignon wine, create a list of all the wines with `variety` column that say Sauvignon, and attach them to the `wines` object from before).

In [35]:
#read the csv data
winemag_csv = pd.read_csv("./data/winemag-data-130k-v2.csv/winemag-data-130k-v2.csv")

#titles: country, description, designation, points, price, province, region1, region2, variety, winery
dfwinemag = winemag_csv[["country", "description", "designation", "points", "price", 
                      "province", "region_1", "region_2", "variety", "winery"]]

#now we have to merge the datasets.
dfwinedata = pd.DataFrame(WineDictionary)

#merge the datasets using pd.merge
wine_df1 = pd.merge(dfwinemag, dfwinedata, how='left', left_on='variety', right_on='Name')

#then drop na axis.
wine_df2 = wine_df1.dropna(axis="rows")

#finally, sort by points
wine_df2 = wine_df2.sort_values(by=["points"])

#print the first five values
wine_df2.head()

,country,description,designation,points,price,province,region_1,region_2,variety,winery,Name,CheeseAndNuts,Desserts,FruitsAndVeggies,HerbsAndSpices,MeatAndPoultry,Sauces,Seafood
109399,US,These vineyards seem to have been selected for...,Selected Vineyards,80,10.0,California,Central Coast,Central Coast,Merlot,Concannon,Merlot,"Parmesan, Pecorino-Romano, chestnuts, walnuts","dark chocolate, berries, fondue","caramelized onions, tomatoes, plums","mint, rosemary, juniper","grilled meats, steak","bolognese, bearnaise","grilled meatier fish, ahi tuna"
123542,US,"A cat-pee, grassy Sauvignon Blanc that's a lit...",Blanche's Vineyard,80,13.0,California,Livermore Valley,Central Coast,Sauvignon Blanc,Cedar Mountain,Sauvignon Blanc,"feta, goat cheese, pine nuts","sorbet, key lime pie, meringue, mango","citrus, green apple, asparagus","chives, tarragon, cilantro","chicken, turkey, pork",citrus and light cream sauces,"fatty white fish, oysters, scallops, lobster,..."
109400,US,"A soft, gluey Pinot Noir, with almost no struc...",Estate Grown,80,18.0,California,Cienega Valley,Central Coast,Pinot Noir,Pietra Santa,Pinot Noir,"goat cheese, brie, walnuts","creme brulee, white chocolate","mushrooms, dried fruits, figs, strawberries","truffle, nutmeg, cinnamon, clove","lamb, sausage, filet mignon, chicken","mushroom sauces, light-medium red sauces","ahi tuna, salmon"
125774,US,The “Reserve” on the label is hardly appropria...,Reserve,80,7.0,California,Monterey,Central Coast,Riesling,Turning Leaf,Riesling,"Havarti, gouda, candied walnuts or pecans","apple pie, caramel sauce","apricots, chili peppers, pears","rosemary, ginger, Thai or Indian spices","smoked sausage, duck, foie gras","BBQ, spicy, chutney","sea bass, trout"
108644,US,"Tasted in early February, this Chardonnay had ...",Kruse Vineyards,80,36.0,California,Central Coast,Central Coast,Chardonnay,Jack Creek,Chardonnay,"mild, semi-soft cheeses with unoaked Chardonn...","banana bread, vanilla pudding","potato, apple, squash, mango","tarragon, sesame, basil","veal, chicken, pork","cream sauces, pesto","halibut, shrimp, crab, lobster"


__C15.__ _(5 pts)_ Now write a function which allows the user to input a list of ingredients and also a choice of ingredient category (such as "Cheese/nuts"), and outputs a list of all wine names that pair with these ingredients. This function won't incorporate the additional ingredients we obtained from the Kaggle dataset.

In [41]:
#define a function
def choose_ingr(ingredient, category):
    
    #let's have an error message just in case
    err = "no wine possible"
    
    #make a new list
    Winelist = []
    
    #for loop: find the name
    for ing in ingredient:
        for i in range(len(wine_df2[category])):
            if ing in wine_df2[category].iloc[i]:
                Winelist.append(wine_df2['Name'].iloc[i])
                
    # from all wines in the list, return the value
    if len(Winelist):
        return Winelist

    # if there is no value, print the error message
    else:
        print(err)

#let's test with walnuts under cheese and nuts. 
# We put this in list(dict.fromkeys()) so as to remove duplicate values. 
option1 = list(dict.fromkeys(choose_ingr(["walnuts"], "CheeseAndNuts")))

#print the types of wines
print(option1)

['Merlot', 'Pinot Noir', 'Riesling', 'Syrah']


__C16.__ _(2 pts)_ This should give you a ton of results! Maybe too many for the function to actually be useful in helping pick a wine. Perhaps there's some disagreement between the datasets. How should the `pairings` and `ingredients` from the scraped data and the Kaggle data each individually be pre-processed as text to obtain a good alignment?

<font color=red>I built the function so that it would not give a ton of results and that the datasets would agree. Combining the datasets from both the Kaggle database and the website is the easiest way to do this. I did this in C14.</font>